In [ ]:
import os
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import pandas as pd

import torch
from Models import *
from functions import *

from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

In [ ]:
num_columns = len(np.genfromtxt('./generated_data/pivot_PM10.csv', delimiter=',', max_rows=1))
columns_to_read_PM = [i for i in range(num_columns) if i != 0]

num_columns = len(np.genfromtxt('./generated_data/pivot_traffic.csv', delimiter=',', max_rows=1))
columns_to_read_traffic = [i for i in range(num_columns) if i != 0]

In [ ]:
x = np.genfromtxt('./generated_data/pivot_traffic.csv', delimiter=',', skip_header=1, usecols=columns_to_read_traffic, dtype=float)
y = np.genfromtxt('./generated_data/pivot_PM10.csv', delimiter=',', skip_header=1, usecols=columns_to_read_PM,  dtype=float)

In [ ]:
print(x.shape)
print(y.shape)
print(x[:,0].shape)
print(y[:,0].shape)

In [ ]:
# import xgboost as xgb

# model = xgb.XGBRegressor()
 
# model.fit(x, y)

# importances = model.feature_importances_
# print(importances)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)

x_test_tensor = torch.tensor(x_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(dataset=train_dataset, batch_size=256, shuffle=True)

test_dataset = TensorDataset(x_test_tensor, y_test_tensor)
test_loader = DataLoader(dataset=test_dataset, batch_size=256, shuffle=True)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
input_size = x.shape[1]
hidden_size = 256
output_size = y.shape[1]
print(input_size)
print(output_size)

# model = MLP(input_size, hidden_size, output_size)
model = MLP_v2(input_size, hidden_size, output_size).to(device)

In [ ]:
#model test
min_mean_abs_error, _, _ = model_test(model, test_loader, device)
print(f'Mean Absolute Error: {min_mean_abs_error:.4f}')

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
num_epochs = 500
mean_absolute_error_list = []

for epoch in range(num_epochs):
    model.train()
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if (epoch) % 20 == 0:
        mean_abs_error, _ , _ = model_test(model, test_loader, device)
        mean_absolute_error_list.append(mean_abs_error)
        if min_mean_abs_error > mean_abs_error:
            min_mean_abs_error = mean_abs_error
            print(f'New min mae: {min_mean_abs_error:.6f}')
            torch.save(model.state_dict(), f"./trained_models/PM10_model_v1.tar")

        print(f'Epoch [{epoch}/{num_epochs}], Loss: {loss.item():.4f}, Mean Absolute Error: {mean_abs_error:.4f}')
        # print(f'Epoch [{epoch}/{num_epochs}], Loss: {loss.item():.4f}')

print(f'Min Mean Absolute Error: {min_mean_abs_error:.4f}')

In [ ]:
model.load_state_dict(torch.load(f"./trained_models/PM10_model_v1.tar", map_location=device))

min_mean_abs_error, _, _ = model_test(model, test_loader, device)
print(f'Mean Absolute Error: {min_mean_abs_error:.4f}')

In [ ]:
#SAVING TO ONNX
#MPL model
import torch.onnx
model.eval()
batch_size = 512
x = torch.randn(batch_size,x.shape[1]).to(device)

output_path = './trained_models/PM10_model_v2.onnx'

# Eksport modelu ONNX z dynamicznym rozmiarem wsadu
torch.onnx.export(model, x, output_path, export_params=True, opset_version=10, do_constant_folding=True,
                    input_names=['input_0'], output_names=['output_0'],
                  dynamic_axes={'input_0': {0: 'batch_size'},'output_0': {0: 'batch_size'}},
                  verbose=True)

In [ ]:
mean_abs_error, preds_list, outs_list = model_test(model, test_loader, device)
print(preds_list.shape)

In [ ]:
#calculate the mean absolute error for every output
from sklearn.metrics import mean_absolute_error

mean_abs_error_list = []
for i in range(output_size):
    mean_abs_error = mean_absolute_error(outs_list[:,i], preds_list[:,i])
    mean_abs_error_list.append(mean_abs_error)

print(mean_abs_error_list)